실습 1: train_test_split (Iris + 로지스틱 회귀)

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target # (150, 4), (150,)

# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
  X, y,
  test_size=0.2, # 20%를 테스트로
  random_state=42, # 재현성
  stratify=y, # 클래스 비율 유지(분류에서 권장)
  shuffle=True
)

# 모델 학습
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# 평가
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print("[Train] acc:", accuracy_score(y_train, y_pred_train))
print("[Test ] acc:", accuracy_score(y_test, y_pred_test))
print("Confusion Matrix (test):", confusion_matrix(y_test, y_pred_test), sep='\n')
print("Classification Report (test):", classification_report(y_test, y_pred_test, target_names=iris.target_names), sep='\n')

[Train] acc: 0.975
[Test ] acc: 0.9666666666666667
Confusion Matrix (test):
[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
Classification Report (test):
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.90      0.95        10
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



실습 B: cross_val_score/cross_validate (Stratified K-Fold)
- 모델 성능을 교차검증(Cross Validation)으로 평가.

- cross_val_score : 단일 지표만 계산
- cross_validate : 여러 지표 동시에 계산

In [3]:
from sklearn.model_selection import StratifiedKFold, cross_validate

# 교차검증 설정 (Stratified: 클래스 비율 유지)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 여러 지표 동시에 측정 : 정확도 accuray, F1 macro
"""
F1 : Precision과 Recall의 조화 평균
* Macro 평균: "클래스별" F1 계산해서 평균. 모든 클래스에 동일한 가중치 적용 -> 소수 클래스 성능도 똑같이 반영
* Micro 평균: "모든" 샘플을 한꺼번에 보고 F1 계산. 큰 클래스의 영향 크게 받음.
"""
scores = cross_validate(
  estimator=LogisticRegression(max_iter=1000), # 사용할 모델
  X=X, y=y, # 사용할 데이터
  cv=cv, # 교차검증 방법(StratifiedKFold)
  scoring=["accuracy", "f1_macro"], # 측정할 지표
  n_jobs=-1, # CPU 병렬 처리 개수. -1이면 모든 코어 사용
  return_train_score=False # 훈련 데이터 점수도 함께 반환할지 여부
) # -> 각 fold마다의 성능 기록 dict 형태로 반환

# 반환 형태 확인
import pandas as pd
df = pd.DataFrame(scores)
print(df.head())

# 결과 요약
acc_mean, acc_std = scores["test_accuracy"].mean(), scores["test_accuracy"].std()
f1m_mean, f1m_std = scores["test_f1_macro"].mean(), scores["test_f1_macro"].std()

print(f"CV Accuracy: {acc_mean:.3f} ± {acc_std:.3f}")
print(f"CV F1-macro: {f1m_mean:.3f} ± {f1m_std:.3f}")
# => 일반화 성능 추정.

   fit_time  score_time  test_accuracy  test_f1_macro
0  0.058604    0.003989       1.000000       1.000000
1  0.066091    0.005275       0.966667       0.966583
2  0.051094    0.006513       0.933333       0.932660
3  0.056609    0.004986       1.000000       1.000000
4  0.056609    0.003989       0.933333       0.933333
CV Accuracy: 0.967 ± 0.030
CV F1-macro: 0.967 ± 0.030


실습 C: 회귀 예시 (California Housing + 선형회귀)

In [4]:
import sklearn
print(sklearn.__version__)

1.7.1


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score

# 데이터 로드
cal = fetch_california_housing() # 단위 : 십만달러($100,000)
Xr, yr = cal.data, cal.target

# 분할
Xr_tr, Xr_te, yr_tr, yr_te = train_test_split(
  Xr, yr, test_size=0.2, random_state=42, shuffle=True
)

# 학습
reg = LinearRegression()
reg.fit(Xr_tr, yr_tr)

# 평가
"""
회귀 모델에서 자주 쓰는 지표
1. R² = 결정계수(Coefficient of Determination)
  : "모델이 데이터를 얼마나 잘 설명하는가"를 0~1 사이 값으로 표현
    -> 즉, 모델이 그냥 평균값 예측보다 얼마나 나은가
  - R² = 1 : 예측 완벽
  - R² = 0 : 평균낸 거랑 차이 없음
  - R² < 0 : 차라리 평균내는 게 나음
2. RMSE(Root Mean Squared Error, 평균제곱근오차)
  : 예측값과 실제값 차이를 오차 크기의 평균로 표현
    ex) 집값 예측에서 RMSE = 50,000 -> 예측이 평균적으로 ±5만달러 정도의 오차가 있다
  
=> R²는 상대적 지표, RMSE는 절대적 지표
따라서, 둘 다 같이 봄
- R²가 높다 == 데이터 패턴 잘 설명한다 (평균보다 얼마나 잘했는지)
- RMSE 낮다 == 실제 오차 크기가 작다 (평균적으로 얼마나 틀렸는지)
"""
yr_pred = reg.predict(Xr_te)
r2 = r2_score(yr_te, yr_pred)
rmse = root_mean_squared_error(yr_te, yr_pred)

print("R²  (test):", r2)
print("RMSE(test):", rmse)

R²  (test): 0.576
RMSE(test): 0.746
